In [7]:
# Import Python Tools:
from IPython.display import display, Latex
import pandas as pd
import numpy as np
from numpy import sqrt, pi, average, cos, sin, tan, arcsin, arccos, arctan, deg2rad, rad2deg

from ASCE import load_combos
from steel_design.AISC import aisc
from concrete_design.material_properties import elastic_modulus, cracked_moment_inertia
from concrete_design.steel_reinforcement import rebar_df as rebar
conc_density = 150.0 #pcf
steel_density = 0.2833 #pci

display(Latex(r"\newpage")) #pagebreak

<IPython.core.display.Latex object>

# Concrete Beam Design
## Inputs

### Materials and Geometry

In [8]:
# Materials
fc = 3000 #psi
fy = 60000 #psi
fyt = 60000 #psi
Es = 29000000 #psi

# Dimensions
ln = 24.5 * 12 #in
b = 18.0 #in
h = 30 #in
cover_t = 1.5 #in
cover_b = 1.5 #in

### Loads

In [9]:
SW = conc_density * b * h / 12**2 #plf
DL = 15 #psf
LL = 40 #psf

LN = ln / 12 #ft
trib = 15 #ft
D = DL * trib / 12 + SW #plf
L = LL * trib / 12 #plf
w_u = load_combos.LRFD(D, L) #plf

Mu = w_u * LN**2 / 8 / 1000 #kip-ft
Vu = w_u * LN / 2 / 1000 #kips
print(f"Mu = {round(Mu / 12, 2)} lb-ft")
print(f"Vu = {round(Vu, 2)} lbs")

Mu = 5.09 lb-ft
Vu = 9.97 lbs


## Strength Design

### $\phi$Mn $\geq$ Mu

#### Steel Input

In [10]:
# Tension Steel
bar = "#6"
Ab = rebar.loc[bar, "Area"]
db = rebar.loc[bar, "Diameter"]
n = 3 #number of bars
layers = 1

d_t = h - cover_b - db/2
print(f"d_t = {round(d_t, 3)} in")

As = n * Ab #in^2
As_ft = As * 12 / b
print(f"As = {round(As, 3)} in^2")

d_t = 28.125 in
As = 1.32 in^2


In [11]:
# Compression Steel
bar = "#6"
Ab_c = rebar.loc[bar, "Area"]
db_c = rebar.loc[bar, "Diameter"]
n = 3 #number of bars
layers = 1

As_c = n * Ab_c #compression reinforcement
d_c = cover_t + db_c/2
print(f"d_c = {round(d_c, 3)} in")

print(f"As = {round(As, 3)} in^2")

d_c = 1.875 in
As = 1.32 in^2


In [12]:
include_compression_steel = "Yes"
if include_compression_steel == "No":
    As_c = 0

#### $\phi$ Factor Calculation

In [13]:
beta_1 = 0.85
a = As * fy / (0.85 * fc * b)
c = beta_1 * a
print(f"a = {round(a, 3)} in, c = {round(c, 3)} in")

eps_ty = fy / Es
eps_cu = 0.003

eps_t = eps_cu * (d_t - c)/c
print(f"eps_t = {round(eps_t, 5)} in/in")

a = 1.725 in, c = 1.467 in
eps_t = 0.05453 in/in


In [14]:
if eps_t >= eps_ty + 0.003:
    phi_b = 0.9
elif eps_t <= eps_ty:
    phi_b = 0.65
else:
    phi_b = 0.65 + 0.35 * (eps_t - eps_ty) / 0.003

f"phi_b = {round(phi_b, 3)}"

'phi_b = 0.9'

#### Flexural Strength

In [15]:
Mn = As * fy * (d_t - a/2) + As_c * fy * (d_t-d_c)
phiMn = phi_b * Mn

print(f"Mu = {round(Mu, 2)} kip-ft")
print(f"phiMn = {round(phiMn / 12 / 1000, 2)} kip-ft")
phiMn > Mu

Mu = 61.06 kip-ft
phiMn = 317.86 kip-ft


True

### $\phi$Vn $\geq$ Vu

#### Steel Input

In [16]:
stirrup_size = "#4"
no_legs = 2

Av = no_legs * rebar.loc[stirrup_size, "Area"]
f"Av = {round(Av, 2)} in^2"

'Av = 0.4 in^2'

#### Shear Strength

In [31]:
Vc = 2 * sqrt(fc) * b * d_t #lbs
f"Vc = {round(Vc / 1000, 2)} kips"

'Vc = 55.46 kips'

In [32]:
s = 12 #in O.C.
Vs = Av * fyt * d_t / s #lbs
f"Vs = {round(Vs / 1000, 2)} kips"

'Vs = 56.25 kips'

In [33]:
phi_v = 0.75
Vn = Vc + Vs
print(f"Vn = {round(Vn / 1000, 2)} kips")
print(f"phiVn = {round(phi_v * Vn / 1000, 2)} kips")

Vn = 111.71 kips
phiVn = 83.78 kips


In [34]:
phi_v * Vn >= Vu

True

### $\phi$ Tn $\geq$ Tu

### $\phi$ Pn $\geq$ Pu

## Deflection

### Short-term Deflection

In [21]:
w_a = load_combos.ASD(D, L)

Ma = w_a * LN**2 / 8 * 12 #lb-in
f"Ma = {round(Ma / 12, 2)} lb-ft"

'Ma = 47363.48 lb-ft'

In [22]:
fr = 7.5 * sqrt(fc) #psi
Ig = b * h**3 / 12 #in^4
print(f"Ig = {round(Ig, 2)} in")
yt = h / 2 #in
Mcr = fr * Ig / yt #lb-in
f"Mcr = {round(Mcr / 12, 2)} lb-ft"

Ig = 40500.0 in


'Mcr = 92428.18 lb-ft'

In [23]:
# Elastic Modulus
Ec = elastic_modulus(fc=fc) #ksi
print(f"Ec = {round(Ec, 2)} ksi")
# Moment of Inertia
Ie = Ig
Icr = cracked_moment_inertia(b=b, h=h, d=d_t, d_c=d_c, Ec=Ec, As=As, As_c=As_c)
print(f"Icr = {round(Icr, 2)} in^4")
if Ma > 2/3*Mcr:
    Ie = Icr / (1 - ((2/3)*Mcr/Ma)**2 * (1 - Icr/Ig))
    print(f"Ie = {round(Ie, 2)} in")
else:
    print("section uncracked")

#Elastic Deflection
defl_elastic = 5 * (w_a/12) * ln**4 / (384 * Ec * 1000 * Ie)
f"Elastic Deflection = {round(defl_elastic, 3)} in"

Ec = 3320.56 ksi
Icr = 6939.51 in^4
section uncracked


'Elastic Deflection = 0.038 in'

### Long-term Deflection

In [24]:
# Deflection Factor
xi = 2.0 #duration factor
rho_c = As_c / (b * d_c)

lmbda_dlt = xi / (1 + 50*rho_c)
f"lmbda_dlt = {round(lmbda_dlt, 3)}"

'lmbda_dlt = 0.677'

In [25]:
# Long-Term Deflection
defl_long = defl_elastic * lmbda_dlt #in
f"Long-Term Deflection = {round(defl_long, 3)} in"

'Long-Term Deflection = 0.026 in'

In [26]:
# Total Deflection
defl_total = defl_elastic + defl_long
f"Total Deflection = {round(defl_total, 3)} in"

'Total Deflection = 0.064 in'

## Minimum Requirements

### Dimension Requirements

#### Minimum Beam Depth

In [27]:
h_min = ln / 16
print(f"h_min (simply supported) = {round(h_min, 2)} in")

h_min = ln / 18.5
print(f"h_min (one end continuous) = {round(h_min, 2)} in")

h_min = ln / 21
print(f"h_min (both ends continuous) = {round(h_min, 2)} in")

h_min = ln / 8
print(f"h_min (cantilever) = {round(h_min, 2)} in")

h_min (simply supported) = 18.38 in
h_min (one end continuous) = 15.89 in
h_min (both ends continuous) = 14.0 in
h_min (cantilever) = 36.75 in


In [28]:
Vu <= phi_v * (Vc + 8*sqrt(fc) * b * d_t)

True

### Minimum Reinforcement

#### Flexural Reinforcement

In [29]:
As_min = max(3*sqrt(fc)/fy, 300/fy) * b * d_t #in^2

f"As,min = {round(As_min, 3)} in^2"

'As,min = 2.531 in^2'

#### Shear Reinforcement

In [30]:
if Vu > phi_v*sqrt(fc)*b*d_t:
    Av_min = max(0.75*sqrt(fc), 50) * b / fyt #in^2
    print(f"Av,min = {round(Av_min, 3)} in^2")
else:
    Av_min = 0
    print("Shear reinforcement not required")

Shear reinforcement not required


#### Torsional Reinforcement